In [ ]:
# import the required libs
import numpy as np
import pylab as pl
import commonUtil

from scipy.interpolate import spline

#To get all query conditions into a dictionary
queryCondition = commonUtil.handle_query_condition()
ISOK_ALL_PARA = queryCondition['ISOK_ALL_PARA']
REPORT_TYPE = queryCondition['REPORT_TYPE']

DB_SET = queryCondition['DB_SET']
START_TIME = queryCondition['START_TIME']
END_TIME = queryCondition['END_TIME']
INTERVAL = queryCondition['INTERVAL']

#To print the error of query conditon, only print once
for err_id in range(len(queryCondition['GENERAL_ERROR'])):
    print queryCondition['GENERAL_ERROR'][err_id]

In [ ]:
if(ISOK_ALL_PARA and (REPORT_TYPE == 'ALL' or REPORT_TYPE == 'RESOURCE')):
    #To declare a Sketchpad as the first graph
    pl.figure(figsize = (24, 6))
    for idx in range(len(DB_SET)):
        #To get data from the target database for assembling a dataframe which will be used for the following graphs
        dataframe_hour = %sql select char(date(collected)) as date, \
                              hour(collected) as hours, \
                              sum(total_cpu_usec_delta) / 1000000.0 as cpu_sec, \
                              sum(logical_reads_delta) as logical_reads, \
                              sum(PHYSICAL_READS_DELTA) as physical_reads, \
                              sum(total_act_time)/(sum(act_aborted_total)+sum(act_completed_total)) as avg_activity_time_msec, \
                              sum(act_completed_total_delta) as activities \
                              from ibm_dsm_views.throughput_all \
                              where dbconn_id = '{DB_SET[idx]}' and collected >= '{START_TIME}' and collected < '{END_TIME}' \
                              group by date(collected), hour(collected) \
                              order by date, hours
        
        if dataframe_hour.empty:
            print 'For resource(CPU Usage): The query result is empty for the database connection ' + DBs[idx] + ', please check your query parameters.\n'
        else:
            #To get the hour data as raw x-axis from the data frame
            x_hour = dataframe_hour['HOURS']
            #To transfer x_hour to a list type
            x_hour_list = list(x_hour)
            #To get the CPU_SEC data as raw y-axis from the data frame
            y_cpu_sec = dataframe_hour['CPU_SEC']
            #To transfer y_cpu_sec to a list type
            y_cpu_sec_new = y_cpu_sec
            #The tick of x-axis and sorted by asc
            x_ticks = []
            #The label of x-axis
            x_ticks_lables = []
            #To get date data for forming date + hour format
            date_all = list(dataframe_hour['DATE'].values)
            
            #To get original datetime used followed judgement
            ori_datetime_str = commonUtil.get_original_datatime_str(date_all, x_hour_list)

            '''
            When the data queried is not equal to the requirement. Filling missing data 
            into the list x_hour_list,date_all and y_ticks.
            '''
            if (len(x_hour_list) < INTERVAL):
                date_all = []
                #clear x_hour_list
                x_hour_list = []
                y_cpu_sec_new = commonUtil.fill_null_data(START_TIME, END_TIME, date_all, x_hour_list, y_cpu_sec, ori_datetime_str)
           
            commonUtil.format_x_axis(date_all, x_hour_list, x_ticks, x_ticks_lables)
            
            #To layout the Sketchpad
            rowNo = len(DB_SET) / 2 + len(DB_SET) % 2    
            arrange = (rowNo * 100) + (2 * 10) + (idx + 1)
            plot = pl.subplot(arrange)
            
            x_label = 'Hours'
            y_label = 'CPU_SEC(s)'
            #To set the title for the first graph
            figure_title='CPU Usage by Hour(' + DB_SET[idx] + ')\n'
            pl.title(figure_title, fontsize = 14, fontweight = 'bold')
            
            #To set x-axis label
            pl.xlabel(x_label, fontsize = 12)
            pl.ylabel(y_label, fontsize = 12)
            #To set grid line style according to your requirement
            pl.grid(True, ls = '--', color = '#a6266e', alpha = 0.05)
            pl.xticks(x_ticks, x_ticks_lables, rotation = 90)
            
            #To mark the the value for per point
            y_cpu_sec_list = list(y_cpu_sec_new)
            for id in range(len(y_cpu_sec_list)):
                if(y_cpu_sec_list[id] == 0.0):#If no data,drawing a empty circle
                    pl.scatter(x_ticks[id], y_cpu_sec_list[id], c = '', marker = 'o', edgecolors = 'r', s = 50)
                else:
                    pl.scatter(x_ticks[id], y_cpu_sec_list[id], c = '#2c628b') 
                    pl.text(x_ticks[id], y_cpu_sec_list[id], '%.2f' % y_cpu_sec_list[id], fontsize = 9)
        
            #x_hour_isSorted=True do nothing, otherwise, handle data sperately by day
            #To smooth the line
            xnew_hour = []
            ynew_cpu_sec = []
            if(len(x_hour_list) >= 3):
                ##In order to smooth the line chart,handle the data further##
                #Expand each x axis data 10 times
                xnew_hour = np.linspace(np.asarray(x_ticks).min(), np.asarray(x_ticks).max(), np.asarray(x_ticks).size * 20) 
                #To handle the data of new y axis data
                ynew_cpu_sec = spline(np.asarray(x_ticks), y_cpu_sec_new, xnew_hour)
                ynew_cpu_sec_list = list(ynew_cpu_sec)

                #If the value of ynew_cpu_sec_list is negative, then change it into positive
                for y_idx in range(len(ynew_cpu_sec_list)):
                    if(ynew_cpu_sec_list[y_idx] < 0.0):
                        ynew_cpu_sec_list[y_idx] = 0.0
                ynew_cpu_sec = np.asarray(ynew_cpu_sec_list)   
                #Fill the gragh according to your requirement
                plot.fill_between(xnew_hour, ynew_cpu_sec, where=(xnew_hour.min()<xnew_hour) & (xnew_hour<xnew_hour.max()), color = '#a6266e', alpha = 0.15)
                #First draw a smooth line chart using xnew_hour and ynew_memory
                plot.plot(xnew_hour, ynew_cpu_sec, color = '#a6266e')

                #To set y-axis value range according to your data
                start_value = ynew_cpu_sec.min()-ynew_cpu_sec.min() / 20
                start_value = float('%0.2f' % start_value)
                end_value = ynew_cpu_sec.max() + ynew_cpu_sec.max() / 20
                end_value = float('%0.2f' % end_value)
                pl.ylim(float('%0.2f' % start_value), float('%0.2f' % end_value))
            else:
                y_cpu_sec = list(map(int, y_cpu_sec_new))
                pl.yticks(y_cpu_sec_new, rotation = 0)
    #Show the first graph
    pl.show()

In [ ]:
if(ISOK_ALL_PARA and (REPORT_TYPE == 'ALL' or REPORT_TYPE == 'RESOURCE')):
    #Declare a Sketchpad as the first graph
    pl.figure(figsize = (24, 4))
    for idx in range(len(DB_SET)):
        #Get data from the target database to assemble a data frame that will be used for the following graphs
        dataframe_hour = %sql select char(date(collected)) as date, \
        hour(collected) as hours, \
        sum(total_cpu_usec_delta) / 1000000.0 as cpu_sec, \
        sum(logical_reads_delta) as logical_reads, \
        sum(PHYSICAL_READS_DELTA) as physical_reads, \
        sum(total_act_time)/(sum(act_aborted_total)+sum(act_completed_total)) as avg_activity_time_msec, \
        sum(act_completed_total_delta) as activities \
        from ibm_dsm_views.throughput_all \
        where dbconn_id = '{DB_SET[idx]}' and collected >= '{START_TIME}' and collected < '{END_TIME}' \
        group by date(collected), hour(collected) order by date, hours
        if dataframe_hour.empty:
            print 'For resource(I/O Consumption): The query result is empty for the database connection ' + DB_SET[idx] + ', please check your query parameters.\n'
        else:
            ###Second graph###
            #Get the hour data as x-axis from the data frame
            x_hour = dataframe_hour['HOURS']
            y_logical_reads = dataframe_hour['LOGICAL_READS']
            y_logical_reads_new = y_logical_reads

            y_physical_reads = dataframe_hour['PHYSICAL_READS']
            y_physical_reads_new = y_physical_reads
            
            log_mil = False
            if y_logical_reads.max() > 1000000:
                log_mil = True
                y_logical_reads = y_logical_reads/1000000
        
            phy_mil = False
            if y_physical_reads.max() > 1000000:
                phy_mil = True
                y_physical_reads = y_physical_reads/1000000
            
            x_ticks = []
            x_ticks_lables = []
            x_hour_list = ''
            x_hour_list = list(x_hour) #here x_hour is a python list
            date_all = list(dataframe_hour['DATE'].values)

            ori_datetime_str = commonUtil.get_original_datatime_str(date_all, x_hour_list)

            '''
            When the data queried is not equal to the requirement. Filling missing data 
            into the list x_hour_list,date_all and y_ticks.
            '''
            if (len(x_hour_list) < INTERVAL):
                y_logical_reads_new = commonUtil.fill_null_data(START_TIME, END_TIME, date_all, x_hour_list, y_logical_reads, ori_datetime_str)
                #clear date_all
                date_all = []
                #clear x_hour_list
                x_hour_list = []
                y_physical_reads_new = commonUtil.fill_null_data(START_TIME, END_TIME, date_all, x_hour_list, y_physical_reads, ori_datetime_str)
            
            if len(x_ticks) == 0:
                commonUtil.format_x_axis(date_all, x_hour_list, x_ticks, x_ticks_lables)
            
            rowNo = len(DB_SET) / 2 + len(DB_SET) % 2    
            arrange = (rowNo * 100) + (2 * 10) + (idx + 1)
            ax1 = pl.subplot(arrange)
            
            y_label = ''
            #Set y-axis label
            if (log_mil==True):
                y_label = 'LOGICAL_READS(mil times)'
            else:
                y_label = 'LOGICAL_READS(times)'
            pl.xlabel(x_label, fontsize = 12)
            pl.ylabel(y_label, fontsize = 12)
            #Set title for the second graph
            figure_title='I/O Consumption by Hour(' + DB_SET[idx] + ')\n'
            pl.title(figure_title, fontsize = 14, fontweight = 'bold')
            pl.xticks(x_ticks, x_ticks_lables, rotation = 90)
            x_hour = np.asarray(x_ticks)
            #Draw the first figure using x_hour and y_logical_reads
            pl.bar(x_hour-0.15, y_logical_reads_new, width = 0.3, color = '#2c628b', alpha = 0.5)
            
            pl.grid(True, ls = '-.',color = '#a6266e', linewidth = '.5', alpha = 0.3)
            #Set ax1 and ax2 to the same x-axis
            ax2 = ax1.twinx()

            #Draw the second graph using x_hour and y_physical_reads
            pl.bar(x_hour+0.15, y_physical_reads_new, width = 0.3, color = '#4c78fb', alpha = 0.6)
            #Set y-axis lab
            
            if (phy_mil==True):
                y_label = 'PHYSICAL_READS(mil times)'
            else:
                y_label = 'PHYSICAL_READS(times)'
            pl.ylabel(y_label, fontsize = 12)
            #Set legend for the ax1 contioner
            ax1.legend(['LOGICAL_READS'], loc = 'upper right', bbox_to_anchor = (0.5, 1.11), ncol = 1)
            #Set legend for the ax2 contioner
            ax2.legend(['PHYSICAL_READS'], loc = 'upper right', bbox_to_anchor = (0.7, 1.11), fancybox = True, shadow = True, ncol = 1)
            #Set grid format
            pl.grid(True, ls = '-.', color = '#a6266e', linewidth = '0.5', alpha = 0.3)
    #Show the first graph
    pl.show()

In [ ]:
if(ISOK_ALL_PARA and (REPORT_TYPE == 'ALL' or REPORT_TYPE == 'RESOURCE')):
    #Declare a Sketchpad as the first graph
    pl.figure(figsize = (24, 3))
    for idx in range(len(DB_SET)):
        #Get data from the target database to assemble a data frame that will be used for the following graphs
        dataframe_hour = %sql select char(date(collected)) as date, \
        hour(collected) as hours, \
        sum(total_cpu_usec_delta) / 1000000.0 as cpu_sec, \
        sum(logical_reads_delta) as logical_reads, \
        sum(PHYSICAL_READS_DELTA) as physical_reads, \
        sum(total_act_time)/(sum(act_aborted_total)+sum(act_completed_total)) as avg_activity_time_msec, \
        sum(act_completed_total_delta) as activities \
        from ibm_dsm_views.throughput_all \
        where dbconn_id = '{DB_SET[idx]}' and collected >= '{START_TIME}' and collected < '{END_TIME}' group by date(collected), hour(collected) order by date, hours
        if dataframe_hour.empty:
            print 'For resource(Average Activity Time): The query result is empty for the database connection ' + DB_SET[idx] + ', please check your query parameters.\n'
        else:
            ###The third graph###
            #Get the hour data as x-axis from the data frame
            x_hour = dataframe_hour['HOURS']
            x_hour_list = list(x_hour)
            y_avg_act = dataframe_hour['AVG_ACTIVITY_TIME_MSEC']
            y_avg_act_new = y_avg_act

            x_ticks = []
            x_ticks_lables = []
            
            date_all = list(dataframe_hour['DATE'].values)

            ori_datetime_str = commonUtil.get_original_datatime_str(date_all, x_hour_list)

            '''
            When the data queried is not equal to the requirement. Filling missing data 
            into the list x_hour_list,date_all and y_ticks.
            '''
            if (len(x_hour_list) < INTERVAL):
                date_all = []
                x_hour_list = []
                y_avg_act_new = commonUtil.fill_null_data(START_TIME, END_TIME, date_all, x_hour_list, y_avg_act, ori_datetime_str)
                
            commonUtil.format_x_axis(date_all, x_hour_list, x_ticks, x_ticks_lables)
            
            rowNo = len(DB_SET) / 2 + len(DB_SET) % 2 
            arrange = (rowNo * 100) + (2 * 10) + (idx + 1)
            plot = pl.subplot(arrange)
            
            #Set title for the second graph
            figure_title='Average Activity Time by Hour(' + DB_SET[idx] + ')\n'
            pl.title(figure_title, fontsize = 14, fontweight = 'bold')
            pl.xticks(x_ticks, x_ticks_lables, rotation = 90)
            x_hour = np.asarray(x_ticks)
            y_label = 'AVG_ACTIVITY_TIME_MSEC'
            pl.ylabel(y_label, fontsize = 12)
            pl.xlabel(x_label, fontsize = 12)
            #Set grid line style according to your requirement
            pl.grid(True, ls = '--', color = '#a6266e', alpha = 0.05)
            #x_hour_isSorted=True do nothing, otherwise, handle data sperately by day
            
            y_avg_act_list = list(y_avg_act_new)
            for id in range(len(y_avg_act_list)):
                if(y_avg_act_list[id] == 0.0):#If no data,drawing a empty circle
                    pl.scatter(x_ticks[id], y_avg_act_list[id], c = '', marker = 'o', edgecolors = 'r', s = 50)
                else:
                    pl.scatter(x_ticks[id], y_avg_act_list[id], c = '#2c628b') 
                    pl.text(x_ticks[id], y_avg_act_list[id], '%.0f' % y_avg_act_list[id], fontsize = 9)
                #x_hour_isSorted=True do nothing, otherwise, handle data sperately by day
            xnew_hour = []
            ynew_cpu_sec = []
            if(len(x_hour_list) >= 3):
                xnew_hour = np.linspace(np.asarray(x_ticks).min(), np.asarray(x_ticks).max(), np.asarray(x_ticks).size * 20) 
                #Handle the data of new y axis data
                ynew_avg_act = spline(np.asarray(x_ticks), y_avg_act_new,xnew_hour)

                ynew_avg_act_list = list(ynew_avg_act)
                #No negative value for y-axis
                for y_idx in range(len(ynew_avg_act_list)):
                    if (ynew_avg_act_list[y_idx] < 0.0):
                        ynew_avg_act_list[y_idx] = 0.0
                ynew_avg_act = np.asarray(ynew_avg_act_list)

                #Fill the gragh according to your requirement
                plot.fill_between(xnew_hour, ynew_avg_act, where=(xnew_hour.min()<xnew_hour) & (xnew_hour<xnew_hour.max()), color = '#a6266e', alpha = 0.15)
                #First draw a smooth line chart using xnew_hour and ynew_memory
                plot.plot(xnew_hour, ynew_avg_act, color = '#a6266e')
                #Set y-axis value range according to your data
                start_value = ynew_avg_act.min() - ynew_avg_act.min() / 2
                start_value = float('%0.2f' % start_value)
                end_value = ynew_avg_act.max() + ynew_avg_act.max() / 2
                end_value = float('%0.2f' % end_value)
                pl.ylim(float('%0.2f' % start_value), float('%0.2f' % end_value))
            else:
                y_avg_act_list = list(map(int, y_avg_act_list))
                pl.yticks(y_avg_act_list, rotation = 0)
    #Show the first graph
    pl.show()

In [ ]:
if(ISOK_ALL_PARA and (REPORT_TYPE == 'ALL' or REPORT_TYPE == 'CPU')):
    #Declare a Sketchpad including two graphs of left and right distribution
    pl.figure(figsize=(24,6))
    for idx in range(len(DB_SET)):
        #Get data from the target database to assemble a data frame that will be used for the following graphs
        dataframe_hour = %sql select char(date(collected)) as date, \
                              hour(collected) as hours, \
                              sum(db2_cpu_user_delta) as db2_cpu_user, \
                              100*sum(db2_cpu_user_delta)/(sum(cpu_user_delta) + sum(cpu_system_delta) + sum(cpu_iowait_delta)+ sum(cpu_idle_delta)) as db2_cpu_user_percent, \
                              sum(db2_cpu_system_delta) as db2_cpu_system,  \
                              100*sum(db2_cpu_system_delta)/(sum(cpu_user_delta) + sum(cpu_system_delta) + sum(cpu_iowait_delta)+ sum(cpu_idle_delta)) as db2_cpu_system_percent, \
                              sum(cpu_user_delta) as cpu_user, \
                              100*sum(cpu_user_delta)/(sum(cpu_user_delta) + sum(cpu_system_delta) + sum(cpu_iowait_delta)+ sum(cpu_idle_delta)) as cpu_user_percent, \
                              sum(cpu_system_delta) as cpu_system, \
                              100*sum(cpu_system_delta)/(sum(cpu_user_delta) + sum(cpu_system_delta) + sum(cpu_iowait_delta)+ sum(cpu_idle_delta)) as cpu_system_percent \
                              from IBM_DSM_VIEWS.THROUGHPUT_SYSTEM \
                              where dbconn_id='{DB_SET[idx]}' and collected >= '{START_TIME}' and collected < '{END_TIME}' \
                              group by date(collected),hour(collected) \
                              order by date,hours
        
        if dataframe_hour.empty:
            print 'For cpu: The query result is empty for the database connection ' + DBs[idx] + ', please check your query parameters.\n'
        else:
            #Get the hour data as x-axis from the data frame
            x_hour = dataframe_hour['HOURS']
            x_hour_list = list(x_hour)
            #Get the db2_cpu_user_percent data as top y-axis of the first graph from the data frame
            y_db2_user = dataframe_hour['DB2_CPU_USER_PERCENT']
            y_db2_user_new = y_db2_user
            #Get the db2_cpu_system_percent data as bottom y-axis of the first graph from the data frame
            y_db2_sys = dataframe_hour['DB2_CPU_SYSTEM_PERCENT']
            y_db2_sys_new = y_db2_sys
            #Get the cpu_user_percent data as top y-axis of the second graph from the data frame
            y_cpu_user = dataframe_hour['CPU_USER_PERCENT']
            y_cpu_user_new = y_cpu_user
            #Get the cpu_system_percent data as bottom y-axis of the second graph from the data frame
            y_cpu_sys = dataframe_hour['CPU_SYSTEM_PERCENT']
            y_cpu_sys_new = y_cpu_sys

            x_ticks = []
            x_ticks_lables = []
           
            date_all = list(dataframe_hour['DATE'].values)

            ori_datetime_str = commonUtil.get_original_datatime_str(date_all, x_hour_list)

            '''
            When the data queried is not equal to the requirement. Filling missing data 
            into the list x_hour_list,date_all and y_ticks.
            '''
            if (len(x_hour_list) < INTERVAL):
                y_db2_user_new = commonUtil.fill_null_data(START_TIME, END_TIME, date_all, x_hour_list, y_db2_user, ori_datetime_str)
                y_db2_sys_new = commonUtil.fill_null_data(START_TIME, END_TIME, date_all, x_hour_list, y_db2_sys, ori_datetime_str)
                y_cpu_user_new = commonUtil.fill_null_data(START_TIME, END_TIME, date_all, x_hour_list, y_cpu_user, ori_datetime_str)
                #clear date_all
                date_all = []
                #clear x_hour_list
                x_hour_list = []
                y_cpu_sys_new = commonUtil.fill_null_data(START_TIME, END_TIME, date_all, x_hour_list, y_cpu_sys, ori_datetime_str)

            if len(x_ticks) == 0:
                commonUtil.format_x_axis(date_all, x_hour_list, x_ticks, x_ticks_lables)
            
            rowNo = len(DB_SET) / 2 + len(DB_SET) % 2    
            arrange = (rowNo * 100) + (2 * 10) + (idx + 1)
            ax1 = pl.subplot(arrange)
            
            #Set the title for the left graph
            figure_title = 'CPU Utilization by Hour(' + DB_SET[idx] + ')\n'
            pl.title(figure_title, fontsize = 14, fontweight = 'bold')
            
            pl.xticks(x_ticks, x_ticks_lables, rotation = 90)
            
            #Set x-axis label
            pl.xlabel(x_label, fontsize = 12)
            #Set y-axis label
            y_label = 'CPU %'
            pl.ylabel(y_label, fontsize = 12)
     
            x_hour = np.asarray(x_ticks)
            #Draw the top y-axis of the upper graph
            pl.bar(x_hour - 0.2, y_db2_user, width = 0.4, align = 'center', color = '#60bdae')
            #Draw the bottom y-axis of the upper graph
            pl.bar(x_hour - 0.2, y_db2_sys, width = 0.4, align = 'center', color = '#a6266e', bottom = y_db2_user)
            #Draw the top y-axis of the lower graph
            pl.bar(x_hour + 0.2, y_cpu_user, width = 0.4, color = '#4c78fb')
            #Draw the bottom y-axis of the lower graph
            pl.bar(x_hour + 0.2, y_cpu_sys, width = 0.4, color = '#2c628b', bottom = y_cpu_user)
            
            pl.yticks(np.arange(0, 101, 10))
            
            #Set the legends for the left graph
            box = ax1.get_position()
            ax1.set_position([box.x0, box.y0 + box.height * 0.1,box.width, box.height * 0.9])
            ax1.legend(['DB2_CPU_USER_PERCENT', 'DB2_CPU_SYSTEM_PERCENT', 'CPU_USER_PERCENT', 'CPU_SYSTEM_PERCENT'], fontsize = 9, loc = 'upper center', bbox_to_anchor = (0.5, 1.06), fancybox = True, shadow = True, ncol = 4)
            #Set grid format for ax1
            pl.grid(True, ls = '-.', color = '#a6266e', linewidth = '0.2', alpha = 0.7)
    #Show all graphs
    pl.show()
    

In [ ]:
if(ISOK_ALL_PARA and (REPORT_TYPE == 'ALL' or REPORT_TYPE == 'MEMORY')):
    #Declare a Sketchpad as the first graph
    pl.figure(figsize = (24, 6))
    for idx in range(len(DB_SET)):
        #Get data from the target database to assemble a data frame that will be used for the following graphs
        dataframe_hour = %sql select char(date(collected)) as date, \
                              hour(collected) as hours, \
                              dec(max(memory_pool_used_gb), 17, 2) as memory_usage_gb \
                              from IBM_DSM_VIEWS.MEM_DB_TOTAL_USED \
                              where dbconn_id = '{DB_SET[idx]}' and collected >= '{START_TIME}' and collected < '{END_TIME}' \
                              group by date(collected), hour(collected) \
                              order by date, hours
        
        if dataframe_hour.empty:
            print 'For memory: The query result is empty for the database connection ' + DBs[idx] + ', please check your query parameters.\n'
        else:
            #Get the hour data as x-axis from the data frame and convert the data into a numpy array
            x_hour= dataframe_hour['HOURS'].values
            x_hour_list = list(x_hour)
            y_memory = dataframe_hour['MEMORY_USAGE_GB'].values
            y_memory_new = y_memory
            #Define a empty var x_ticks to store x-axis ticks(marks)
            x_ticks = []
            #Define a empty var x_ticks_lables to restore x-axis labels
            x_ticks_lables = []
            date_all = list(dataframe_hour['DATE'].values)
            
            ori_datetime_str = commonUtil.get_original_datatime_str(date_all, x_hour_list)

            '''
            When the data queried is not equal to the requirement. Filling missing data 
            into the list x_hour_list,date_all and y_ticks.
            '''
            if (len(x_hour_list) < INTERVAL):
                #clear date_all
                date_all = []
                #clear x_hour_list
                x_hour_list = []
                y_memory_new = commonUtil.fill_null_data(START_TIME, END_TIME, date_all, x_hour_list, y_memory, ori_datetime_str)

             
            commonUtil.format_x_axis(date_all, x_hour_list, x_ticks, x_ticks_lables)
                
            rowNo_mry = len(DB_SET) / 2 + len(DB_SET) % 2
            arrange_mry = (rowNo_mry * 100) + (2 * 10) + (idx + 1)
            plot = pl.subplot(arrange_mry)
            
            #Get the data as the label of x-axis
            #Set the title for the graph
            figure_title='Memory Usage by Hour(' + DB_SET[idx] + ')\n'
            pl.title(figure_title, fontsize = 14, fontweight = 'bold')
            pl.xticks(x_ticks, x_ticks_lables, rotation = 90)
            #Set x-axis label
            pl.xlabel(x_label, fontsize = 12)
            #Set y-axis label
            y_label = 'MEMORY_USAGE(GB)'
            pl.xlabel(y_label, fontsize = 12)
            #Set grid line style according to your requirement
            pl.grid(True, ls = '--', color = '#2c628b', alpha = 0.05)
            
            y_memory_list = list(y_memory_new)
            for id in range(len(y_memory_list)):
                if(y_memory_list[id] == 0.0):#If no data,drawing a empty circle
                    pl.scatter(x_ticks[id], y_memory_list[id], c = '',marker = 'o', edgecolors = 'r', s = 50)
                else:
                    pl.scatter(x_ticks[id], y_memory_list[id], c = '#2c628b') 
                    pl.text(x_ticks[id], y_memory_list[id], '%.2f' % y_memory_list[id], fontsize = 9)
            
            #Below two variable is used for storeing magnified data
            xnew_hour = []
            ynew_memory = []
            #if the size of data is greater than 2,showing a line not a scatter
            #otherwise, a scatter graph will be showed
            if(len(x_hour_list) >= 3):
                #Get the memory_usage_gb data as y-axis from the data frame and convert the data into a numpy array
                ##In order to smooth the line chart,handle the data further##
                #Expand each x axis data 20 times
                xnew_hour = np.linspace(np.asarray(x_ticks).min(), np.asarray(x_ticks).max(), np.asarray(x_ticks).size*20) 
                #Handle the data of new y axis data
                ynew_memory = spline(np.asarray(x_ticks), y_memory_new, xnew_hour)
                
                ynew_memory_list = list(ynew_memory)
                #No negative value for y-axis
                for y_idx in range(len(ynew_memory_list)):
                    if (ynew_memory_list[y_idx] < 0.0):
                        ynew_memory_list[y_idx] = 0.0
                ynew_memory = np.asarray(ynew_memory_list)
                
                #Fill the gragh according to your requirement
                plot.fill_between(xnew_hour,ynew_memory, where=(xnew_hour.min()<xnew_hour) & (xnew_hour<xnew_hour.max()), color = '#2c628b', alpha = 0.09)
               
                #First draw a smooth line chart using xnew_hour and ynew_memory
                plot.plot(xnew_hour, ynew_memory, color = '#2c628b')

                #Set y-axis value range according to your data
                start_value=ynew_memory.min() - ynew_memory.min() / 2
                start_value=float('%0.2f' % start_value)
                end_value=ynew_memory.max() + ynew_memory.max() / 2
                end_value=float('%0.2f' % end_value)
                if((start_value == 0.0) and (end_value == 0.0)):
                    y_memory_list = list(map(int, y_memory_list))
                    pl.yticks(y_memory_list, rotation = 0)
                else:
                    pl.ylim(float('%0.2f' % start_value), float('%0.2f' % end_value))
            else:
                y_memory_list = list(map(int, y_memory_list))
                pl.yticks(y_memory_list,rotation=0)
                
    #Show the graph
    pl.show()